# Groundedness Evaluator

### Getting Started
This sample demonstrates how to use the Groundedness evaluator to assess whether AI-generated responses are grounded in the provided context. The evaluator supports multiple input formats including:
- Simple response and context evaluation
- Query, response, and context evaluation
- Agent responses with tool calls (file_search)
- Multi-turn conversations

This notebook uses consistent examples with the ToolCallAccuracyEvaluator for better comparison across evaluators.

Before you begin:
```bash
pip install azure-ai-evaluation
```
Set these environment variables with your own values:
1) **MODEL_DEPLOYMENT_NAME** - The deployment name of the model for this AI-assisted evaluator
2) **AZURE_OPENAI_ENDPOINT** - Azure OpenAI Endpoint to be used for evaluation
3) **AZURE_OPENAI_API_KEY** - Azure OpenAI Key to be used for evaluation
4) **AZURE_OPENAI_API_VERSION** - Azure OpenAI API version to be used for evaluation

## What is Groundedness?

The Groundedness evaluator assesses the correspondence between claims in an AI-generated response and the source context. It ensures that responses are substantiated by the provided context, preventing hallucinations and unsupported claims.

**Key Points:**
- Even factually correct responses are considered ungrounded if they can't be verified against the provided context
- Essential for RAG (Retrieval-Augmented Generation) applications
- Helps ensure AI responses are trustworthy and verifiable

**Scoring:** Groundedness scores range from 1 to 5, with:
- **1**: Completely ungrounded - no claims supported by context
- **2**: Mostly ungrounded - few claims supported
- **3**: Partially grounded - some claims supported
- **4**: Mostly grounded - most claims supported
- **5**: Fully grounded - all claims supported by context

## Groundedness Evaluator Input Requirements

The Groundedness evaluator supports multiple input formats:

1. **Basic Context Evaluation:**
   - `response`: The AI response to evaluate (str)
   - `context`: The source context/documents (str)
   - `query`: Optional query for enhanced evaluation (str)

2. **Agent Tool Evaluation:**
   - `query`: The user query (str)
   - `response`: Agent response with tool calls (List[dict])
   - `tool_definitions`: Available tools, only file_search supported (List[dict])

3. **Conversation Evaluation:**
   - `conversation`: Multi-turn conversation with context (Conversation object)

### Initialize Groundedness Evaluator

In [ ]:
import os
from azure.ai.evaluation import GroundednessEvaluator, AzureOpenAIModelConfiguration
from pprint import pprint

# Configure the model
model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
)

# Initialize the evaluator
groundedness_evaluator = GroundednessEvaluator(model_config=model_config)

## Sample Evaluations

### Response and Context as Strings (str)

In [ ]:
# Example of a well-grounded response using weather context
context = """Current weather data for Seattle shows rainy conditions with a temperature of 14°C. 
The forecast indicates overcast skies with light precipitation typical for Pacific Northwest weather. 
Humidity is at 85% with winds from the southwest at 12 mph. Visibility is reduced to 8 miles due to rain."""

response = "The current weather in Seattle is rainy with a temperature of 14°C. It's typical Pacific Northwest weather for this time of year with overcast skies and light precipitation."

result = groundedness_evaluator(response=response, context=context)
pprint(result)

### Example of Ungrounded Response

In [ ]:
# Example of ungrounded response with unsupported claims
context = """Current weather data for Seattle shows rainy conditions with a temperature of 14°C. 
The forecast indicates overcast skies with light precipitation typical for Pacific Northwest weather."""

response = "The current weather in Seattle is rainy with a temperature of 14°C. The city is experiencing its wettest month in 50 years, and the mayor has declared a weather emergency due to flooding concerns."

result = groundedness_evaluator(response=response, context=context)
pprint(result)

### Query, Response, and Context as Strings (str)

In [ ]:
query = "How is the weather in Seattle?"

context = """Weather report for Seattle, Washington: Currently experiencing rainy weather with temperature at 14°C. 
Overcast conditions with light rain are expected to continue. The current conditions are typical for the Pacific Northwest region 
during this season. Wind speed is moderate at 12 mph from southwest direction."""

response = "The weather in Seattle is rainy with a temperature of 14°C. These are typical Pacific Northwest conditions with overcast skies."

result = groundedness_evaluator(response=response, context=context, query=query)
pprint(result)

### Query as String (str), Response as List[dict], Tool Definitions as List[dict]

In [ ]:
query = "Can you get me the current weather information for Seattle?"

# Agent response with file_search tool call
agent_response = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_filesearch_weather_123",
                "name": "file_search",
                "arguments": {"query": "current weather Seattle temperature conditions"}
            }
        ]
    },
    {
        "role": "tool",
        "tool_call_id": "call_filesearch_weather_123", 
        "content": [
            {
                "type": "tool_result",
                "tool_result": {
                    "content": "Seattle weather report: Currently rainy with temperature of 14°C. Overcast skies with light precipitation. Typical Pacific Northwest weather with 85% humidity and southwest winds at 12 mph."
                }
            }
        ]
    },
    {
        "role": "assistant",
        "content": "Based on the weather data, Seattle is currently experiencing rainy weather with a temperature of 14°C. The conditions include overcast skies and light precipitation, which is typical for the Pacific Northwest."
    }
]

tool_definitions = [
    {
        "name": "file_search",
        "description": "Search through uploaded files to find relevant information",
        "parameters": {
            "type": "object", 
            "properties": {
                "query": {"type": "string", "description": "Search query"}
            }
        }
    }
]

result = groundedness_evaluator(
    query=query, 
    response=agent_response, 
    tool_definitions=tool_definitions
)
pprint(result)

### Conversation as Dict with Context and Messages

In [ ]:
conversation = {
    "context": "Weather data shows Seattle currently has rainy conditions at 14°C with overcast skies. London shows cloudy weather at 8°C with partly cloudy conditions. Both cities are experiencing typical seasonal weather patterns.",
    "messages": [
        {
            "role": "user",
            "content": "Can you check the weather in Seattle for me?"
        },
        {
            "role": "assistant",
            "content": "According to the current weather data, Seattle is experiencing rainy conditions with a temperature of 14°C and overcast skies."
        },
        {
            "role": "user",
            "content": "How does that compare to London?"
        },
        {
            "role": "assistant",
            "content": "London is currently cloudier but drier than Seattle, with a temperature of 8°C and partly cloudy conditions. Seattle is warmer but rainier at 14°C."
        },
        {
            "role": "user", 
            "content": "Can you email me a summary of both cities' weather?"
        },
        {
            "role": "assistant",
            "content": "I can provide you with a weather summary: Seattle has rainy weather at 14°C with overcast skies, while London has partly cloudy conditions at 8°C. However, I would need email access to send this information to you."
        }
    ]
}

result = groundedness_evaluator(conversation=conversation)
pprint(result)